## Thử nghiệm khả năng nhận diện tin giả của các model LLMs

In [140]:
import sys
import os
import requests
import json

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)

# Bây giờ import
from CrewAI.tools.check import (
    check_url_virustotal, parse_vt_result_for_display,
    check_email_validity, parse_email_result,
    check_phone_validity, parse_phone_result
)

from CrewAI.tools.search_googleapi import search_google_api

In [21]:
API_URL = "http://172.17.128.1:1234/v1/chat/completions"
MODEL_NAME = "qwen2.5-7b-instruct-1m"  # Hoặc "llama3-8b-instruct", "mistral-7b-instruct",...

In [139]:
def extract_fields(data: dict):
    summary = data.get('summary', '').strip()
    keywords_list = data.get('keywords', [])
    request_user = data.get('request_user', '').strip()
    email = data.get('email', '').strip()
    phone = data.get('phone', '').strip()
    url = data.get('url', '').strip()

    keywords = ", ".join(str(k) for k in keywords_list if isinstance(k, str))

    return summary, keywords, request_user, email, phone, url

def llm_extract(input_text: str, api_url: str, model_name: str):
    messages = [
        {
            "role": "system",
            "content": (
                "Bạn là trợ lý AI. **Chỉ trả về đúng kết quả duy nhất dưới dạng JSON thuần túy, không kèm thêm bất kỳ lời giải thích hay đoạn văn nào khác.**\n"
                "Output structure phải theo dạng:\n"
                "{\n"
                '  "summary": "...",\n'
                '  "keywords": [...],\n'
                '  "request_user": "...",\n'
                '  "emails": [...],\n'
                '  "phones": [...],\n'
                '  "urls": [...]\n'
                "}\n"
                "Không thêm code block, không thêm dấu ` ``` `, không thêm bất kỳ ký tự nào khác."
            )
        },
        {
            "role": "user",
            "content": input_text
        }
    ]

    response = requests.post(
        api_url,
        headers={"Content-Type": "application/json"},
        json={
            "model": model_name,
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 1024,
            "stream": False
        }
    )

    if response.status_code != 200:
        raise RuntimeError(f"LLM request failed: {response.status_code} {response.text}")

    result = response.json()
    content = result['choices'][0]['message']['content']

    try:
        json_str = content.strip()
        if json_str.startswith("{") and "'" in json_str:
            json_str = json_str.replace("'", '"')
        parsed = json.loads(json_str)
    except json.JSONDecodeError:
        parsed = {}

    # Fallback nếu summary hoặc keywords rỗng
    summary = parsed.get("summary", "").strip()
    keywords = parsed.get("keywords", [])
    request_user = parsed.get("request_user", "")
    emails = parsed.get("emails", [])
    phones = parsed.get("phones", [])
    urls = parsed.get("urls", [])

    if not summary:
        summary = input_text
    if not keywords:
        keywords = list({w.lower() for w in input_text.split() if len(w) > 3})

    # Lấy 1 email, phone, url duy nhất dưới dạng string hoặc "" nếu không có
    email = emails[0] if isinstance(emails, list) and len(emails) > 0 else ""
    phone = phones[0] if isinstance(phones, list) and len(phones) > 0 else ""
    url = urls[0] if isinstance(urls, list) and len(urls) > 0 else ""

    return {
        "summary": summary,
        "keywords": keywords,
        "request_user": request_user,
        "email": email,
        "phone": phone,
        "url": url
    }


In [ ]:
def build_checks_summary(url=None, email=None, phone=None):
    parts = []

    if url:
        url_result = check_url_virustotal(url)
        check_url = parse_vt_result_for_display(url_result)
        parts.append(f"Kết quả kiểm tra URL: {check_url}")

    if email:
        mail_result = check_email_validity(email)
        check_mail = parse_email_result(mail_result)
        parts.append(f"Kết quả kiểm tra Mail: {check_mail}")

    if phone:
        phone_result = check_phone_validity(phone)
        check_phone = parse_phone_result(phone_result)
        parts.append(f"Kết quả kiểm tra Phone: {check_phone}")

    return "\n".join(parts)

def build_news_summary(keywords):
    news_df = search_google_api(keywords)

    lines = []
    for _, row in news_df.iterrows():
        title = row.get('title', '')
        content = row.get('content', '')
        lines.append(f"- Tiêu đề: {title}\n  Nội dung: {content[:300]}...")
    return "\n".join(lines)

def verify_scam(
    summary, url, email, phone, news_df, api_url, model_name):
    check_summary = build_checks_summary(url, email, phone)
    news_summary = build_news_summary(news_df)

    prompt_system = f"""
    Bạn là trợ lý AI chuyên xác minh tính xác thực của thông tin, giúp phân loại yêu cầu của người dùng có phải tin tức giả mạo, lừa đảo hay đáng nghi vấn không.

    Thông tin đầu vào gồm:

    1. Tóm tắt yêu cầu (summary):
    {summary}

    2. Kết quả kiểm tra URL, Email, Số điện thoại (nếu có):
    {check_summary}

    3. Các tin tức tham khảo liên quan được tìm thấy trên internet (có thể không hoàn toàn liên quan):
    {news_summary}

    Dựa vào các thông tin trên, vui lòng phân loại yêu cầu của người dùng thuộc loại nào trong ba nhóm sau:

    - ✅ An toàn: thông tin tin cậy, không có dấu hiệu lừa đảo hay giả mạo
    - ❌ Lừa đảo: thông tin có dấu hiệu giả mạo, lừa đảo rõ ràng
    - ⚠️ Đáng ngờ: thông tin có dấu hiệu nghi vấn, cần thận trọng

    Chỉ trả lời đúng theo định dạng sau, không thêm giải thích hay bình luận khác:

    Kết luận: <✅ An toàn | ❌ Lừa đảo | ⚠️ Đáng ngờ>

    messages = [
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": ""}
    ]

    response = requests.post(
        api_url,
        headers={"Content-Type": "application/json"},
        json={
            "model": model_name,
            "messages": messages,
            "temperature": 0,
            "max_tokens": 20000,
            "stream": False
        }
    )

    if response.status_code != 200:
        raise RuntimeError(f"LLM request failed: {response.status_code} {response.text}")

    result = response.json()
    content = result['choices'][0]['message']['content'].strip()
    return content

In [142]:
input_text = 'Bạn đã trúng thưởng 1 chiếc điện thoại IPHONE16 Plus, hãy nhấp vào link sau để nhận thưởng: trungthuong.com.vn'

result = llm_extract(input_text, API_URL, MODEL_NAME)

summary, keywords, request_user, email, phone, url = extract_fields(result)

print("Summary:", summary)
print("Keywords:", keywords)
print("Request User:", request_user)
print("Emails:", email)
print("Phones:", phone)
print("URLs:", url)

Summary: Thông báo trúng thưởng giả mạo
Keywords: trúng,  Smartphone, nhận, link, gửi
Request User: 
Emails: 
Phones: 
URLs: trungthuong.com.vn


In [143]:
output = verify_scam(summary, url, email, phone, keywords, API_URL, MODEL_NAME)

print(output)

Kết luận: ❌ Lừa đảo
